In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Workbench를 활용한 모델 생성 및 모델 배포 실습 

## 개요

본 튜토리얼은 Iris 데이터를 활용하여 VerTex AI Workbench 환경에서 학습하여 모델을 생성하고 생성된 모델을 팀별 또는 개인별 저장소에 업로드하여 버전, 히스토리 등이  관리되게 하는 환경을 실습합니다.

<img src="./imgs/workbench.png" alt="Workbench 구조" width="400"/>

### 실습 목표

The objective of this notebook is to create, deploy and serve a custom classification model on Vertex AI. This notebook focuses more on deploying the model than on the design of the model itself. 


This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI Models
- Vertex AI Endpoints

The steps performed include:

- Train a model that uses flower's measurements as input to predict the class of iris.
- Save the model and its serialized pre-processor.
- Build a FastAPI server to handle predictions and health checks.
- Build a custom container with model artifacts.
- Upload and deploy custom container to Vertex AI Endpoints.

### 데이터 셋

본 튜토리얼은 R.A. Fisher의 Iris 데이터셋을 활용하여 모델을 생성합니다. 
 tutorial uses R.A. Fisher's Iris dataset, a small dataset that is a popular choice for trying out machine learning techniques. Each instance has four numerical features, which are different measurements of a flower, and a target label that
marks it as one of three types of iris: Iris setosa, Iris versicolour, or Iris virginica.

### 샘플 학습에 필요한 패키지 설치 및 적용

샘플 학습에 필요한 패키지 설치를 위한 다음의 코드를 실행합니다. 
실행이 완료되면 설치된 패키지 적용을 위해서 다음과 같은 메시지가 보입니다. "OK" 버튼을 클릭하여 설치된 패키지를 적용합니다. 

<img src="./imgs/kernel_restart.png" alt="설치된 패키지 적용" width="400"/>

In [ ]:
%%writefile requirements.txt
joblib~=1.0
numpy~=1.20
scikit-learn~=0.24
google-cloud-storage>=1.26.0,<2.0.0dev

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

# Required in Docker serving container
! pip3 install -U {USER_FLAG} -r requirements.txt -q

# For local FastAPI development and running
! pip3 install -U {USER_FLAG} "uvicorn[standard]>=0.12.0,<0.14.0" fastapi~=0.63 -q

# Vertex SDK for Python
! pip3 install -U {USER_FLAG} google-cloud-aiplatform -q

# 설치된 패키지 반영
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 시작 하기 전

#### 구글 클라우드 서비스 (Google Cloud Storage 등)를 사용하기 위한 환경 설정

팀별 또는 개인별로 생성된 스토리지나 모델을 저장하기 위한 저장소에 접근하기 위한 기본 환경을 설정합니다.

```다음 코드는 향후 Google Cloud 기반에서 Jupyter Notebook을 활용할 때 재 사용할 수 있는 코드입니다.```

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

## 학습을 위한 샘플 데이터가 저장되어 있는 스토리지 명
SAMPLE_DATA_STORAGE_NAME = "hmg-vertexai-data"

## 프로젝트 ID 가져오기
shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
PROJECT_ID = shell_output[0]
print("Project ID: ", PROJECT_ID)

REGION = "[your-region]"

if REGION == "[your-region]":
    REGION = "asia-northeast3"

print("Region : " , REGION)

! gcloud config set project {PROJECT_ID}

## 생성된 모델 등의 파일을 저장하기 위한 스토리지 생성
BUCKET_NAME = PROJECT_ID
BUCKET_URI = f'gs://{BUCKET_NAME}'

! gsutil mb -l {REGION} {BUCKET_URI}

#### 실습에 사용될 공통 변수 설정

Set a name for the following resources:

`MODEL_ARTIFACT_DIR` - Folder directory path to your model artifacts within a Cloud Storage bucket, for example: "my-models/fraud-detection/trial-4"

`REPOSITORY` - Name of the Artifact Repository to create or use.

`IMAGE` - Name of the container image that is pushed to the repository.

`MODEL_DISPLAY_NAME` - Display name of Vertex AI Model resource.

In [ ]:
MODEL_ARTIFACT_DIR = "[your-artifact-directory]"  
REPOSITORY = "[your-repository-name]" 
IMAGE = "[your-image-name]"  
MODEL_DISPLAY_NAME = "[your-model-display-name]" 
DATA_DIR = "[your-data-directory]"
ROOT_DIR = "[your-root-directory]"
MODEL_STORE_DIR = "[your-model-store-directory]"

# Set the defaults if no names were specified
if MODEL_ARTIFACT_DIR == "[your-artifact-directory]":
    MODEL_ARTIFACT_DIR = "custom-container-prediction-model"

if REPOSITORY == "[your-repository-name]":
    REPOSITORY = "custom-container-prediction"

if IMAGE == "[your-image-name]":
    IMAGE = "sklearn-fastapi-server"

if MODEL_DISPLAY_NAME == "[your-model-display-name]":
    MODEL_DISPLAY_NAME = "sklearn-custom-container"
    
if DATA_DIR == "[your-data-directory]":
    DATA_DIR = "data"

if ROOT_DIR == "[your-root-directory]":
    ROOT_DIR = "app"
    
if MODEL_STORE_DIR == "[your-model-store-directory]":
    MODEL_STORE_DIR = "model"

#### 라이브러리 임포트

In [ ]:
from google.cloud import aiplatform

#### Vertex AI SDK for Python 초기화

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

#### 모델 생성을 위한 디렉토리 생성

In [ ]:
%mkdir {ROOT_DIR}

## Data 디렉토리 생성
%mkdir -p {ROOT_DIR}/{DATA_DIR}

## Model 저장을 위한 디렉토리 생성
%mkdir -p {ROOT_DIR}/{MODEL_STORE_DIR}

#### 학습 데이터 다운로드

모델 학습에 필요한 iris.csv 파일을 샘플 데이터가 저장되어 있는 스토리지에서 다운로드 받습니다.

In [ ]:
! gsutil cp gs://hmg-vertexai-data/iris.csv {ROOT_DIR}/{DATA_DIR}/

#### 학습 및 모델 생성


In [ ]:
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

filePath = f"{ROOT_DIR}/{DATA_DIR}/iris.csv"
df = pd.read_csv(filePath)
df.info()
df.head()

df.Species.value_counts()

setosa_df = df[df.Species == 'setosa']
ax = setosa_df["Sepal.Length"].plot(kind='hist')
setosa_df["Sepal.Length"].plot(kind='kde', ax=ax, secondary_y=True, figsize=(8,4))

X_train,X_test,Y_train,Y_test = train_test_split(df.iloc[:,0:4],df['Species'],test_size=0.2)

model = GaussianNB()

model.fit(X_train,Y_train)

joblib.dump(model, "model.joblib")

#### 생성된 모델 클라우드 스토리지에 저장하기

In [ ]:
! gsutil cp {ROOT_DIR}/{MODEL_STORE_DIR}/model.joblib {BUCKET_URI}/{MODEL_ARTIFACT_DIR}

#### 셍성된 모델을 Google Cloud의 Model Repository 에 저장하기

생성된 모델을 구글 클라우드 Vertex AI 서비스 중 하나인 Model Repository에 저장합니다.
저장을 위해서는 일련의 패키징 작업이 필요합니다. 다음 코드는 패키징 작업 후 모델에 저장하는 코드 샘플입니다.

Model Repository는 학습하여 생성된 모델을 저장하여 허가된 사용자에게 공유하여 사용가능하며,
버전 관리가 가능하며, 예측을 위해 원격으로 요청이 가능합니다.

In [ ]:
%%writefile Dockerfile

FROM tiangolo/uvicorn-gunicorn-fastapi:python3.9

COPY ./app /app
COPY requirements.txt requirements.txt

RUN pip install -r requirements.txt

In [ ]:
!gcloud artifacts repositories create {REPOSITORY} \
    --repository-format=docker \
    --location={REGION}

!gcloud builds submit --region={REGION} --tag={REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}

model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=f"{BUCKET_URI}/{MODEL_ARTIFACT_DIR}",
    serving_container_image_uri=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}",
)